In [14]:
'''
Create a dataset of shortest paths between all destinations
'''

import requests
import urllib.request
import pathlib
import os
import shutil
import subprocess
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import nearest_points, cascaded_union
import math
import swifter

import json
import geojson
import gdal
import h3

import networkit as nk

import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_IN = PATH_ROOT + '/data/02_processed/'
PATH_OUT = PATH_ROOT + '/data/03_paths/'

# constants
CONST_speed_day_km = 40
CONST_slope_effect_multiplier = 2
CONST_graph_detail_lvl = 8
CONST_distance_bridge = 10 / 111
CONST_river_value = 9999

# critical slope algorithm
CONST_critical_slope = 4

# for tobler application
CONST_elevation_coefficient = 1

def slope_coeff(slope):
    #return math.exp(-3.5 * abs(slope + 0.05)) ** CONST_elevation_coefficient
    #rise = (value_to - value_from) / 1000
    #slope = rise / hex_distance
    return 1 / (1 + ((abs(slope) * 100) / CONST_critical_slope) ** 2)

def calculate_path_time(value_from, value_to, dist):
    rise = (value_to - value_from) / 1000
    slope = rise / dist
    slope_c = slope_coeff(slope)
    #print(slope_c)
    time = CONST_speed_day_km * abs(slope_c ** CONST_slope_effect_multiplier)

    el_dist = math.sqrt((rise ** 2) + (dist ** 2))
    path_time = (el_dist / time)

    #print(value_to, value_from, slope, time, el_dist, path_time)
    
    return path_time

#print(calculate_path_time(100, 300, 2))
#print(calculate_path_time(100, 200, 1))
print(calculate_path_time(50, 200, 10))
print(calculate_path_time(200, 50, 10))

crs4326 = {'init': 'epsg:4326'}
hex_distance = 2 * h3.edge_length(CONST_graph_detail_lvl)
print(hex_distance)
CONST_hex_dist = hex_distance / 111

'''
Load datasets
'''

destinations = gpd.read_file(PATH_IN + 'destinations.geojson')

rivers_df = gpd.read_file(PATH_IN + 'rivers.geojson') 
rivers = cascaded_union(
    [river['geometry'] for ri, river in rivers_df.iterrows()]
)

bridges_df = gpd.read_file(PATH_IN + 'bridges.geojson')
bridges = cascaded_union(
    [bridge['geometry'].buffer(hex_distance / 222) for bi, bridge in bridges_df.iterrows()]
)

bbox = gpd.read_file(PATH_IN + 'bbox.geojson') 
bb_xy = bbox.total_bounds

bounds_json = dict(
    geojson.Polygon(
        [[
            [bb_xy[0], bb_xy[1]],
            [bb_xy[0], bb_xy[3]],
            [bb_xy[2], bb_xy[3]],
            [bb_xy[2], bb_xy[1]],
            [bb_xy[0], bb_xy[1]],
        ]]
    ))


0.3252929369373301
0.3252929369373301
0.922709368


In [15]:
# load elevation
elevation = gdal.Open(PATH_IN + 'elevation.tif')

band = elevation.GetRasterBand(1)

cols = elevation.RasterXSize
rows = elevation.RasterYSize

transform = elevation.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

elevation_data = band.ReadAsArray(0, 0, cols, rows)

# check whether there is river or a bridge
def path_value_point (hex_center):
    #return elevation_point(hex_center)
    #on_river = hex.intersects(rivers)
    on_river = hex_center.distance(rivers) < CONST_hex_dist / 2
    if on_river: 
        on_bridge = hex_center.distance(bridges) < CONST_hex_dist / 2
        if not on_bridge:
            value = CONST_river_value
        else:
            value = elevation_point(hex_center)
    else:
        value = elevation_point(hex_center)
    return value

# get elevation value for the given point
def elevation_point (point):
    row = int((yOrigin - point.y ) / pixelHeight)
    col = int((point.x - xOrigin) / pixelWidth)
    
    if elevation_data.shape[0] > row and elevation_data.shape[1] > col:
        return int(elevation_data[row][col])
    else:
        return CONST_river_value

hex_ids = list(h3.polyfill(bounds_json, CONST_graph_detail_lvl))

def create_hex(hex_id):
    center = Point(h3.h3_to_geo(hex_id))
    value = path_value_point(center)

    return {
        "h3id": hex_id,
        #"id": hex_id,
        "value": value,
        "center": wkt.loads(wkt.dumps(center.simplify(0.0, preserve_topology=True), rounding_precision=4)),
        #"geometry": Polygon(h3.h3_to_geo_boundary(hex_id))
    }

hexes_list = Parallel(n_jobs=num_cores)(
    delayed(create_hex)(hex_id) for hex_id in hex_ids
)

print('hexes list ready')

hex_values = {}
hex_centers = {}

for hi, hex in enumerate(hexes_list):
    hex_centers[hex['h3id']] = hex['center']
    #if hex['value'] != CONST_river_value:
    hex_values[hex['h3id']] = hex['value']


print('no hexes', len(hex_values))
print('river hexes', len(hexes_list) - len(hex_values))

hexes list ready
no hexes 1711237
river hexes 0


In [16]:
#gpd.GeoDataFrame(hexes_list)
elevation_df = gpd.GeoDataFrame(hexes_list, geometry="center", crs="epsg:4326")
elevation_df['center'] = elevation_df.swifter.apply(
      lambda x: wkt.loads(wkt.dumps(x['center'].simplify(0.0, preserve_topology=True), rounding_precision=5)),
      axis=1
  )
elevation_df[['center', 'value']].to_file(PATH_OUT + 'elevation.shp', driver="ESRI Shapefile")

Pandas Apply: 100%|██████████| 1711237/1711237 [03:33<00:00, 8005.64it/s]


In [19]:
'''
Calculate hex id for each destination
'''

destinations_hexes = {}
for di, destination in destinations.iterrows():
    closest_node = False
    distance_min = 2000
    point = destination['geometry']

    for hi, hex_id in enumerate(hex_centers) :
        if hex_values[hex_id] != CONST_river_value:
            center = hex_centers[hex_id]

            distance_node = center.distance(point)
            
            if (distance_node < distance_min):
                closest_node = hex_id
                distance_min = distance_node

            if (distance_node < CONST_hex_dist ):
                break
    
    #print(distance_min)
    destinations_hexes[destination['name']] = closest_node

öhmen
region2                   Nordböhmen
region3                   Nordböhmen
geometry         POINT (14.76 50.77)
Name: 34, dtype: object
POINT (15.89 49.09) id                                  183
name            Jaroměřice nad Rokytkou
no_alle                               5
no_activity                           2
no_reise                              2
no_bi                                 1
no_gg                                 0
x                                 49.09
y                                 15.89
prazision                             1
noten          Jaromerice nad Rokytnou?
region1                          Mähren
region2                    Znaimer Teil
region3                    Znaimer Teil
geometry            POINT (15.89 49.09)
Name: 35, dtype: object
POINT (16.87 48.5) id                                 184
name                      Jedenspeigen
no_alle                              4
no_activity                          2
no_reise                             2
n

ValueError: Null geometry supports no operations

In [ ]:
'''
Create graph
'''
g = nk.Graph(directed=False, weighted=True)
hex_nodes = {}
node_hexes = {}
edges = {}

for hi, hex_id in enumerate(hex_values):
    hex_nodes[hex_id] = g.addNode()
    node_hexes[hex_nodes[hex_id]] = hex_id

for hi, hex_from_id in enumerate(hex_values):
    value_from = hex_values[hex_from_id]

    neighs = h3.get_h3_unidirectional_edges_from_hexagon(hex_from_id)
    g.addNode()

    for neigh in neighs:
        hex_to_id = h3.get_destination_h3_index_from_unidirectional_edge(neigh)
        if hex_to_id in hex_values:

            # non-directed graph
            if hex_from_id > hex_to_id:
                value_to = hex_values[hex_to_id]
                #print(value_to)
                g.addEdge(hex_nodes[hex_from_id], hex_nodes[hex_to_id], calculate_path_time(value_from, value_to, hex_distance))

In [ ]:
'''
Calculate shortest paths between destinations
'''

paths = []
for di1, d1 in enumerate(destinations.itertuples(), 1):
    #if di1 == 0:
    d1_name = d1.name
    d1_hex = destinations_hexes[d1_name]
    node_from = hex_nodes[d1_hex]

    dj_paths = nk.distance.Dijkstra(g, node_from, True, False)
    dj_paths.run()

    for di2, d2 in enumerate(destinations.itertuples(), 1):

        # non-directional graph
        if di1 < di2:
        #if di2 < 15:
            d2_name = d2.name
            d2_hex = destinations_hexes[d2_name]
            node_to = hex_nodes[d2_hex]

            try:

                dist = dj_paths.distance(node_to)
                path = dj_paths.getPath(node_to)

                path_hexes = LineString([hex_centers[node_hexes[i]] for i in path])
                paths.append(
                    {
                        "from": d1_name,
                        "to": d2_name,
                        "dist": dist,
                        "geometry": path_hexes
                    }
                )
                paths.append(
                    {
                        "from": d2_name,
                        "to": d1_name,
                        "dist": dist,
                        "geometry": path_hexes
                    }
                )
            except:
                pass

In [ ]:
paths_df = gpd.GeoDataFrame(paths, crs="epsg:4326")
paths_df.to_file(PATH_OUT + 'paths.shp', driver="ESRI Shapefile", encoding="utf-8")
#paths_df.to_file(PATH_OUT + 'paths.geojson', driver="GeoJSON")

In [ ]:
paths_text = "".join([line.strip() for line in open(PATH_OUT + 'paths.geojson')])
paths_dict = json.loads(paths_text)

In [ ]:
len(paths_dict['features'])


origins = {}
for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from'] 
    origins[orig] = {'origin': orig}

for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from']
    dest = feat['properties']['to']
    dist = feat['properties']['dist']
    origins[orig][dest] = dist

dist_df = pd.DataFrame(origins.values())

dist_df.set_index('origin').to_csv(PATH_OUT + 'dist_m.csv')

